## World Resources Institute's thermal power plant physical climate vulnerability factors

### CITATION
Tianyi Luo, Lihuan Zhou, James Falzon, Yan Cheng, Giulia Christianson, Yili Wu and Amir Habchi, Assessing Physical Climate Risks for the European Bank for Reconstruction and Development's Power Generation Project Investment Portfolio, World Resources Institute, Working paper, doi:10.46830/wriwp.21.00060.

### LINK
https://www.wri.org/research/assessing-physical-climate-risks-european-bank-power-portfolio

### NOTES
Data from Table B1 in Appendix B converted into CSV format in raw.csv. Ingested into vulnerability model as standard json.


In [1]:
import json
import pandas

data = {"items": []}

df = pandas.read_csv("raw.csv")
df["Asset Type"] = [
    (x if pandas.isna(y) else (x + "/" + y)).replace(" ", "")
    for (x, y) in zip(df["Turbine Type"], df["Cooling Technology"])
]
df.drop(columns=["Turbine Type", "Cooling Technology"], inplace=True)

hazard_set = set(df["Hazard Type"])
for hazard_type in hazard_set:
    selection = df.loc[df["Hazard Type"] == hazard_type]
    asset_set = set(selection["Asset Type"])
    for asset_type in asset_set:
        sub_selection = selection.loc[selection["Asset Type"] == asset_type]
        threshold_set = set(sub_selection["Threshold Type"])
        for threshold_type in threshold_set:
            item = sub_selection.loc[sub_selection["Threshold Type"] == threshold_type]
            if (
                len(set(item["Threshold Unit"])) == 1
                and len(set(item["Vulnerability Type"])) == 1
                and len(set(item["Vulnerability Unit"])) == 1
            ):
                data["items"].append(
                    {
                        "asset_type": asset_type.replace(" ", ""),
                        "event_type": hazard_type.replace(" ", ""),
                        "location": "Global",
                        "intensity": list(item["Threshold"].values),
                        "intensity_units": item["Threshold Unit"]
                        .values[0]
                        .replace(" ", ""),
                        "impact_type": item["Vulnerability Type"]
                        .values[0]
                        .replace(" ", ""),
                        "impact_units": item["Vulnerability Unit"]
                        .values[0]
                        .replace(" ", ""),
                        "impact_mean": list(item["Vulnerability"].values),
                        "impact_std": [],
                    }
                )

with open(
    "WRI thermal power plant physical climate vulnerability factors.json", "w"
) as f:
    vulnerability_json = json.dumps(data, sort_keys=True, indent=4)
    f.write(vulnerability_json)